In [1]:
import pandas as pd
import scipy as sp
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats.anova import anova_lm
import scipy.stats as stats

In [3]:
path = pd.read_csv("~/.../pathways_out/path_abun_relab_description.csv",index_col=0)
path = path.transpose()
metadata = pd.read_csv("~/.../metadata_all.csv",index_col=0)
path_sample = metadata.index.intersection(path.index)
path_name = path.loc["description"]
path_metadata = pd.concat([metadata.loc[path_sample],path.loc[path_sample]],sort = True, axis =1 )


In [4]:
def path_stat(result):
    #there are a ton of stuff can be simplified or optimized for more pathral use: I can load the file and filter groups based on given names
    #i can set an argument to determine the numbers of groups, factors and determine which stats should be run
    res_sca=result  
    paths = res_sca.columns.tolist()[4:]
    res_stat = pd.DataFrame()
    res_mc = pd.DataFrame()

    for path in paths: 
        test = res_sca.loc[:,["Group", "Y-A","Time","Location",path]]
        test.columns = ["Group","YA","time","site","value"]
        test.loc[:,"value"]=test.loc[:,"value"].astype(float)
        model = ols('value ~ C(YA)*C(time)*C(site)', test).fit()
        aov_table = anova_lm(model, typ=1)
        res_stat=res_stat.append(aov_table["PR(>F)"].transpose()[0:7])
        mc = statsmodels.stats.multicomp.MultiComparison(test['value'],test['Group'])
        mc_BH_table = mc.allpairtest(stats.ttest_ind,method='fdr_bh')[0]
        mc_table=pd.DataFrame(mc_BH_table.data[1:],columns=mc_BH_table.data[0])
        res_mc = res_mc.append(mc_table["pval_corr"].transpose())
    res_mc.index = paths
    res_stat.index = paths
    res_mc.columns = mc_table["group1"]+"_"+mc_table["group2"]
    res_stats = pd.concat([res_stat,res_mc],axis =1)
    return res_stats #,res_sca_sig

In [5]:
path_abun_stat = path_stat(path_metadata)

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:325: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha


In [6]:
path_abun_stat_name = pd.concat([path_name,path_abun_stat],axis = 1)

In [8]:
path_abun_stat_name.to_csv("~/.../path_abun_relab_all_name_stat.csv")